In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
#read data
routes = pd.read_csv('routes_final.csv')
routes.head()

,Route ID,Start City,Start Date,Greedy Route,Optimal Route,Greedy Price,Optimal Price,Target Audience,Region,Subregion,Month,Trip ID,Greedy Distance
0,1,PARIS,2025-07-01,"['BARCELONA', 'IBIZA', 'LISBON', 'SPLIT', 'BER...","['BARCELONA', 'LISBON', 'BERLIN', 'IBIZA', 'SP...",722.01,660.24,YOUNG,EUROPE,WEST,JULY,A,5243.77
1,2,PARIS,2025-08-01,"['BARCELONA', 'IBIZA', 'LISBON', 'SPLIT', 'BER...","['BARCELONA', 'LISBON', 'BERLIN', 'IBIZA', 'SP...",949.49,836.46,YOUNG,EUROPE,WEST,AUGUST,A,5243.77
2,3,PARIS,2025-07-01,"['NICE', 'ROME', 'MAJORCA', 'IBIZA', 'MYKONOS']","['MAJORCA', 'MYKONOS', 'IBIZA', 'ROME', 'NICE']",772.90,670.43,YOUNG,EUROPE,WEST,JULY,B,4250.02
3,4,PARIS,2025-08-01,"['NICE', 'ROME', 'MAJORCA', 'IBIZA', 'MYKONOS']","['MAJORCA', 'IBIZA', 'MYKONOS', 'ROME', 'NICE']",1338.91,707.53,YOUNG,EUROPE,WEST,AUGUST,B,4250.02
4,5,TAMPA,2025-07-01,"['FORT LAUDERDALE', 'MIAMI', 'WASHINGTON', 'NE...","['FORT LAUDERDALE', 'NEW YORK CITY', 'MIAMI', ...",1501.52,887.77,YOUNG,USA,EAST,JULY,C,2468.10


In [18]:
#compare greedy price with optimal price

print(f"Number of instances: {routes.shape[0]}")

greedy_greater_count = routes[routes['Greedy Price'] > routes['Optimal Price']].shape[0]

print(f"Greedy Price was higher than the Optimal Price in {greedy_greater_count} instances.")



Number of instances: 22
Greedy Price was higher than the Optimal Price in 22 instances.


In [24]:
#checking how many times the optimal route is the reverse of the greedy route

def is_reverse(route1, route2):
    route1_list = eval(route1)
    route2_list = eval(route2)
    return route1_list == route2_list[::-1]

routes['Is Reverse'] = routes.apply(lambda row: is_reverse(row['Greedy Route'], row['Optimal Route']), axis=1)
reverse_cases = routes[routes['Is Reverse']]

print(f"Total cases where Optimal Route is the reverse of Greedy Route: {len(reverse_cases)}\n")
for _, row in reverse_cases.iterrows():
    start_city = row['Start City']
    greedy_route = " → ".join(eval(row['Greedy Route']))
    optimal_route = " → ".join(eval(row['Optimal Route']))
    
    print(f"Case:")
    print(f"Route ID        {row['Route ID']}")
    print(f"Target Audience {row['Target Audience']}")
    print(f"Region          {row['Region']}")
    print(f"Subregion       {row['Subregion']}")
    print(f"Month           {row['Month']}\n")
    print(f"Greedy Route    {start_city} → {greedy_route} → {start_city}")
    print(f"Optimal Route   {start_city} → {optimal_route} → {start_city}")


Total cases where Optimal Route is the reverse of Greedy Route: 1

Case:
Route ID        21
Target Audience ELDER
Region          USA
Subregion       WEST
Month           JULY

Greedy Route    SANTA BARBARA → PALM SPRINGS → JACKSON HOLE, WYOMING → PORTLAND, OREGON → NAPA VALLEY → CHICAGO → SANTA BARBARA
Optimal Route   SANTA BARBARA → CHICAGO → NAPA VALLEY → PORTLAND, OREGON → JACKSON HOLE, WYOMING → PALM SPRINGS → SANTA BARBARA


In [31]:
#checking when the same trip but in different months has different routes

#grouping the data by 'Trip ID' to compare the 'Optimal Route' across different months
trip_groups = routes.groupby('Trip ID')

same_routes = 0
different_routes = 0

#iterate through each trip group to compare 'Optimal Route'
for trip_id, group in trip_groups:
    unique_routes = group['Optimal Route'].unique()
    
    if len(unique_routes) == 1:  # All routes are the same
        same_routes += 1
        
        # Print the details for the group
        print(f"Trip ID: {trip_id}")
        print(f"Optimal Route: {unique_routes[0]}")
        print(f"Target Audience {group['Target Audience'].iloc[0]}")
        print(f"Region          {group['Region'].iloc[0]}")
        print(f"Subregion       {group['Subregion'].iloc[0]}")
        print()
    else:
        different_routes += 1  # There are different routes

# Print the summary
print(f"\nTrips with the same 'Optimal Route': {same_routes}")
print(f"Trips with different 'Optimal Route': {different_routes}")





Trip ID: A
Optimal Route: ['BARCELONA', 'LISBON', 'BERLIN', 'IBIZA', 'SPLIT']
Target Audience YOUNG
Region          EUROPE
Subregion       WEST

Trip ID: G
Optimal Route: ['PORTO', 'BARCELONA', 'VENICE,ITALY', 'AIX-EN-PROVENCE', 'ROME']
Target Audience ELDER
Region          EUROPE
Subregion       A


Trips with the same 'Optimal Route': 2
Trips with different 'Optimal Route': 9
